In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
import pm4py
import pandas as pd

In [2]:
path = "./"

In [3]:
# These lines reads the log file
log_file_name = '2020WithBeginAndEnd.xes'
log_file_name_output = '2020WithBeginAndEndDuplicated.xes'
log = xes_importer.apply(path + log_file_name)

c:\workspace\.venv\lib\site-packages\pm4py\util\dt_parsing\parser.py:77: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  "ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11")


parsing log, completed traces ::   0%|          | 0/1998 [00:00<?, ?it/s]

In [4]:
df = pm4py.convert_to_dataframe(log)

c:\workspace\.venv\lib\site-packages\pm4py\utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [5]:
def rename_df_node(df, df_relation, rename_source_node, df_element_new_value, 
                  event_classifier = ['case:concept:name', 'lifecycle:transition'],
                  event_ordered_column = ['time:timestamp'],
                  event_attribute_column = 'concept:name'):
    import pandas as pd
    import itertools

    # for clas in event_classifier:
    grouped = df.groupby(event_classifier)
    filtered_rows_idx = []

    # iterate through each group
    for name, group in grouped:
        # sort the group by timestamp
        group = group.sort_values(event_ordered_column)

        # iterate through each row in the group
        for i in range(1, len(group)):
            # check if the previous task is "add item" and the current task is "Pack item"
            if group.iloc[i-1][event_attribute_column] == df_relation[0] and group.iloc[i][event_attribute_column] == df_relation[1]:
                # add the index values of the two rows to the filtered list
                if rename_source_node:
                    filtered_rows_idx.append(group.iloc[i-1:i].index.tolist())
                else:
                    filtered_rows_idx.append(group.iloc[i:i+1].index.tolist())

    filtered_rows_idx = list(itertools.chain(*filtered_rows_idx))

    df.loc[filtered_rows_idx,event_attribute_column] = df_element_new_value

    return df


In [6]:
# for assumption 5
df = rename_df_node(df, ('register resut', 'send rejection letter'), rename_source_node=False, 
                  df_element_new_value='send rejection letter 4 UC', 
                  event_classifier = ['case:concept:name', 'lifecycle:transition'],
                  event_ordered_column = ['time:timestamp'],
                  event_attribute_column = 'concept:name')

In [7]:
# for assumption 6
df = rename_df_node(df, ('inform customer', 'END'), rename_source_node=True, 
                  df_element_new_value='inform customer 2 END', 
                  event_classifier = ['case:concept:name', 'lifecycle:transition'],
                  event_ordered_column = ['time:timestamp'],
                  event_attribute_column = 'concept:name')

In [8]:
log = pm4py.convert_to_event_log(df)

c:\workspace\.venv\lib\site-packages\pm4py\utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [9]:
pm4py.write_xes(log, log_file_name_output)

exporting log, completed traces ::   0%|          | 0/1998 [00:00<?, ?it/s]